In [1]:
import calendar

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
mpl.rcParams["figure.figsize"] = 16, 9

In [3]:
#data = (pd.read_csv("/home/leyht/Downloads/untitled folder/{}.csv".format(k), header=0, index_col=0,
#                    parse_dates=True) for k in calendar.month_name[1:])
#data = pd.concat(data, keys=range(1,13), names=("month", "date"))
#data.to_hdf("data/futureswise.h5", "data", complevel=9, complib="bzip2")

In [4]:
data = pd.read_hdf("data/futureswise.h5")
termstructure = pd.read_csv("data/8_m_settle.csv", usecols=range(1,10), parse_dates=True,
                            header=0, index_col=0, na_values=0)

In [5]:
for year in data:
    yint = int(year)
    # This internally raises an exception but it's just some localization problem.
    # The resulting data seems correct.
    data[year].index = pd.MultiIndex(levels=[data[year].index.levels[0],
                                             data[year].index.levels[1].map(lambda x: pd.Timestamp(yint, x.month, x.day))],
                                     labels=data[year].index.labels)

Exception ignored in: 'pandas._libs.tslib._localize_tso'
Traceback (most recent call last):
  File "pandas/_libs/tslib.pyx", line 4332, in pandas._libs.tslib._get_dst_info (pandas/_libs/tslib.c:72500)
  File "pandas/_libs/tslib.pyx", line 1425, in pandas._libs.tslib._get_utcoffset (pandas/_libs/tslib.c:27137)
AttributeError: 'Int64Index' object has no attribute 'utcoffset'
Exception ignored in: 'pandas._libs.tslib._localize_tso'
Traceback (most recent call last):
  File "pandas/_libs/tslib.pyx", line 4332, in pandas._libs.tslib._get_dst_info (pandas/_libs/tslib.c:72500)
  File "pandas/_libs/tslib.pyx", line 1425, in pandas._libs.tslib._get_utcoffset (pandas/_libs/tslib.c:27137)
AttributeError: 'Int64Index' object has no attribute 'utcoffset'
Exception ignored in: 'pandas._libs.tslib._localize_tso'
Traceback (most recent call last):
  File "pandas/_libs/tslib.pyx", line 4332, in pandas._libs.tslib._get_dst_info (pandas/_libs/tslib.c:72500)
  File "pandas/_libs/tslib.pyx", line 1425, in 

In [6]:
# Concatenate the columns to get a large series of spread prices.
spreads = pd.concat((data[year].dropna() for year in data))

In [7]:
date_mask = spreads.index.droplevel(0).isin(termstructure.index)
filtered_spreads = spreads.where(date_mask).dropna()

In [8]:
# These are the spread prices loaded from file.
filtered_spreads[:,"2017-04-27"]

month
6    -0.200
7     0.450
8    -0.300
9     0.375
10    0.100
11    0.200
12   -0.800
dtype: float64

In [9]:
# These are spread prices calculated from the term structure.
# Why are these different from each other?
termstructure.loc["2017-04-27"].aggregate(lambda x: [2*x[i] - x[i-1] - x[i+1] for i in range(1,7)])

[-0.27000000000000135,
 0.46000000000000085,
 -0.3100000000000005,
 0.41999999999999815,
 0.050000000000004263,
 0.24999999999999645]